In [1]:
def frame_to_time(frame_number, fps=25):
    return frame_number / fps


In [3]:
def time_to_frame(time_seconds, fps=25):
    return int(time_seconds * fps)


In [5]:
def get_keypoints_for_segment(sentence_name, start_time, end_time, json_dir, fps=25):
    start_frame = time_to_frame(start_time, fps)
    end_frame = time_to_frame(end_time, fps)

    keypoints_sequence = []

    for frame_num in range(start_frame, end_frame + 1):
        json_file = os.path.join(json_dir, f"{sentence_name}_{frame_num:012d}.json")

        if os.path.exists(json_file):
            with open(json_file, 'r') as f:
                frame_data = json.load(f)
                keypoints_sequence.append(frame_data)
        else:
            print(f"Warning: Missing JSON file for frame {frame_num}")

    return keypoints_sequence


In [7]:
def find_matching_json_files(sentence_name, json_dir):
    patterns = [
        f"{sentence_name}_*.json",
        f"{sentence_name.replace('--', '_')}*.json",
        f"*{sentence_name.split('_')[1]}*.json"
    ]

    for pattern in patterns:
        files = glob.glob(os.path.join(json_dir, pattern))
        if files:
            return files
    return []


In [9]:
def handle_missing_frames(keypoints_sequence, expected_length):
    if len(keypoints_sequence) < expected_length:
        # Simple interpolation: duplicate the last frame
        last_frame = keypoints_sequence[-1]
        while len(keypoints_sequence) < expected_length:
            keypoints_sequence.append(last_frame)
    return keypoints_sequence


In [17]:
import pandas as pd

df = pd.read_csv('D://Stage PFA 2A//how2sign_realigned_test.csv', sep="\t")
print(df.head())


      VIDEO_ID               VIDEO_NAME    SENTENCE_ID  \
0  -fZc293MpJk  -fZc293MpJk-1-rgb_front  -fZc293MpJk_0   
1  -fZc293MpJk  -fZc293MpJk-1-rgb_front  -fZc293MpJk_2   
2  -fZc293MpJk  -fZc293MpJk-1-rgb_front  -fZc293MpJk_3   
3  -fZc293MpJk  -fZc293MpJk-1-rgb_front  -fZc293MpJk_4   
4  -fZc293MpJk  -fZc293MpJk-1-rgb_front  -fZc293MpJk_5   

               SENTENCE_NAME  START_REALIGNED  END_REALIGNED  \
0  -fZc293MpJk_0-1-rgb_front             0.26           6.79   
1  -fZc293MpJk_2-1-rgb_front             7.27          20.30   
2  -fZc293MpJk_3-1-rgb_front            21.25          25.51   
3  -fZc293MpJk_4-1-rgb_front            27.75          44.64   
4  -fZc293MpJk_5-1-rgb_front            46.68          52.44   

                                            SENTENCE  
0                                                Hi!  
1  The aileron is the control surface in the wing...  
2  By moving the stick, you cause pressure to inc...  
3  The elevator is the part that moves with th

In [19]:
df.head()

,VIDEO_ID,VIDEO_NAME,SENTENCE_ID,SENTENCE_NAME,START_REALIGNED,END_REALIGNED,SENTENCE
0,-fZc293MpJk,-fZc293MpJk-1-rgb_front,-fZc293MpJk_0,-fZc293MpJk_0-1-rgb_front,0.26,6.79,Hi!
1,-fZc293MpJk,-fZc293MpJk-1-rgb_front,-fZc293MpJk_2,-fZc293MpJk_2-1-rgb_front,7.27,20.30,The aileron is the control surface in the wing...
2,-fZc293MpJk,-fZc293MpJk-1-rgb_front,-fZc293MpJk_3,-fZc293MpJk_3-1-rgb_front,21.25,25.51,"By moving the stick, you cause pressure to inc..."
3,-fZc293MpJk,-fZc293MpJk-1-rgb_front,-fZc293MpJk_4,-fZc293MpJk_4-1-rgb_front,27.75,44.64,The elevator is the part that moves with the s...
4,-fZc293MpJk,-fZc293MpJk-1-rgb_front,-fZc293MpJk_5,-fZc293MpJk_5-1-rgb_front,46.68,52.44,"Therefore, it's either going uphill, downhill,..."


In [ ]:
import os
import json

json_dir = 'D:/Stage PFA 2A/test_2D_keypoints/openpose_output/json'

for idx, row in df.iterrows():
    sentence_name = row['SENTENCE_NAME']
    start_time = row['START_REALIGNED']
    end_time = row['END_REALIGNED']
    text = row['SENTENCE']

    keypoints = get_keypoints_for_segment(sentence_name, start_time, end_time, json_dir)
    # Traiter les keypoints et les segments vidéo
